In [1]:
#FILE CON I MIEI TENTATIVI DI PREPROCESSING SUL DATABASE DI ANOBii

#IMPORT
from pyspark.sql.functions import rand
from pyspark.sql.functions import lower
from pyspark.sql.types import IntegerType, StringType
#from lightfm import LightFM
from pyspark.sql.functions import row_number, lit, dense_rank
from pyspark.sql import functions as F, Window
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

#CARICAMENTO DATAFRAME BASE
file = "/data/SMARTDATA/books/anobii/author_item.csv"
DFauthorbooks = spark.read.csv(file,header=True) #MOSTRA I LIBRI RELATIVI AGLI AUTORI
#DFauthorbooks.show(1, False)
#LIBRI DEGLI AUTORI

file = "/data/SMARTDATA/books/anobii/author_display.csv"
DFdisplay = spark.read.csv(file,header=True) #MOSTRA GLI AUTORI DEL DATABASE
#DFdisplay.show(1, False)
#DFauthorbooks_withnames = DFauthorbooks.join(DFdisplay, DFauthorbooks.author_id == DFdisplay.author_id).drop(DFdisplay.author_id)
#AUTORI

file = "/data/SMARTDATA/books/anobii/language_mapping.csv"
DFlanguages = spark.read.csv(file,header=True)
#DFlanguages.show()

#Dobbiamo tenere solo i libri con language=11 #FILTRO LINGUAGGIO

file = "/data/SMARTDATA/books/anobii/item_recommend.csv"
DFrecommends = spark.read.csv(file,header=True)
#DFrecommends.show(1, False)
#Questa tabella contiene gli ISBN riferiti all'item Id (quindi item_id non è unico?)

file = "/data/SMARTDATA/books/anobii/item.csv"
DFitems_anobii = spark.read.csv(file,header=True) #MOSTRA I LIBRI DEL DATABASE

file = "/data/SMARTDATA/books/anobii/item_comment_vote.csv" #MOSTRA INFORMAZIONI SUL VOTO DEGLI UTENTI
DFvotes = spark.read.csv(file,header=True)
#DFvotes.show(50, False)

file = "/data/SMARTDATA/books/anobii/item_comment_feedback.csv" #MOSTRA IL VOTO MEDIO
DFfeedback = spark.read.csv(file,header=True)
#DFfeedback.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_recommend.csv" #MOSTRA I COMMENTI DEGLI UTENTI RELATIVI A UN LIBRO
DFcomments = spark.read.csv(file,header=True)
#DFcomments.show(50, False)

file = "/data/SMARTDATA/books/anobii/person_item_progress.csv" #MOSTRA INFORMAZIONI SULLE RACCOMANDAZIONI
DFprogress = spark.read.csv(file,header=True)
#DFprogress.show(50, False)

file = "/data/SMARTDATA/books/anobii/link_person_item_comment.csv" #MOSTRA INFORMAZIONI VARIE QUALI LA PRESENZA DI
#SPOILER IN UN COMMENTO O LA PRESENZA DI PROFANITà
DFinfo = spark.read.csv(file,header=True)

file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
DFstars = spark.read.csv(file,header=True)

#Aggiungiamo encrypt_item_id e genere al dataframe dei libri
#DFjoinbookstars_and_authors_nooneword.filter(DFjoinbookstars_and_authors_nooneword.book_id=="1981748").show()
#Per aggiungere encrypted_item_id, apriamo il dataframe dei dati nuovi di anobii
file = file = "/data/SMARTDATA/books/anobii_2021/sql/item.csv"
DFnew_data = spark.read.csv(file, header=True)
DFnew_data = DFnew_data.filter(DFnew_data.language=="11")
#DFnew_data.show(1, False)
#DFnew_data.filter(DFnew_data.item_id == "1981748").show()
#Lo schema di questo dataset è enorme! Almeno sono riuscito a trovare il path.

file = "anobii_genres/genres5.csv"
DFgenres = spark.read.csv(file, header=True)
#DFgenres.filter(DFgenres.itemid == "1981748").show(10, False)

#DFmanifestations_with_encrypt.filter(DFmanifestations_with_encrypt.book_id == "672529").show()
#FUNZIONI
def title_and_subtitle(line): #Usata per splittare titolo e sottotitolo nel database delle biblioteche e renderli più simili ad anobii
    if len(line.title.split(" : ")) > 1: #Il libro possiede un sottotitolo
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
        
        return Row(line.manifestation_id_new, line.patron_id_md5, title, sub_title, line.author, 4, "None", "None", "None", line.ISBNISSN_new,"biblioteche") #NE APPROFITTO ANCHE PER INSERIRE VOTO E CAMPI PER IL MERGE (PER ORA NULL)
    else: #Il libro non possiede un sottotitolo
        return Row(line.manifestation_id_new, line.patron_id_md5, line.title, "None", line.author, 4, "None", "None", "None", line.ISBNISSN_new, "biblioteche")

def create_user_dictionary(rdd): #Used to assign an integer id to each user of the rdd (to get the rows of the CSR)
    rdd = rdd.map(lambda x: (str(x.person_id), list(x))).sortByKey()
    user_dictionary = rdd.countByKey()
    i = 0
    for key in user_dictionary.keys():
        user_dictionary[key] = i
        i += 1
    return user_dictionary

def create_book_dictionary(rdd): #Used to assign an integer id to each book of the rdd (to get the column of the CSR)
    rdd = rdd.map(lambda x: (x.book_id, list(x))).sortByKey()
    book_dictionary = rdd.countByKey()
    i = 0
    for key in book_dictionary.keys():
        book_dictionary[key] = i
        i += 1
    return book_dictionary

def addDataType1(line):
    return Row(line.item_id, line.person_id, line.title, str(line.sub_title), line.item_review, line.total_review, line.average_rating, str(line.total_votes), line.isbn, "anobii")


def addRowColumnId(line): #Funzione usata per aggiungere gli indici per ogni libro e utente
    book_number = book_dictionary[line.book_id] 
    user_number = user_dictionary[line.person_id]
    
    return Row(line.person_id, line.title, str(line.sub_title), line.item_review, line.data_type, line.book_id, line.author, line.genre, line.encrypt_item_id, line.total_votes_or_loans, line.average_rating, line.total_review, line.isbn, str(user_number), str(book_number))

def title_and_subtitle_books(line): #FUNZIONE USATA PER DIVIDERE TITOLO E SOTTOTITOLO NEL DATASET DEI LIBRI DELLE BIBLIOTECHE E PER ELIMINARE EDITION_DATE ED EDITION_LANGUAGE
    if len(line.title.split(" : ")) > 1:
        title = line.title.split(" : ")[0]
        sub_title = line.title.split(" : ")[1]
    else:
        title = line.title
        sub_title = "None"
    #"title", "sub_title", "author", "publisher", "book_id"
    return Row(title, sub_title, line.author, line.publisher, line.manifestation_id_new, line.ISBNISSN_new)

In [2]:
#FILTRAGGIO 1: linguaggio italiano
#Mi basterà filtrare da item.csv tutte le tuple con language diverso da 11.

DFfilteredlanguageitems = DFitems_anobii.filter(DFitems_anobii.language == "11")
#DFfilteredlanguageitems.show(20, False)

#FILTRAGGIO 2: solo "hardcover" e "paperback"

DFfilteredbindingitems = DFfilteredlanguageitems.filter((DFfilteredlanguageitems.binding == "Paperback") | (DFfilteredlanguageitems.binding == "Hardcover"))
#DFfilteredbindingitems.show(20, False)

#FILTRAGGIO 3: eliminare i fumetti (per quanto possibile)
#Userò come parole chiave personaggi famosi Disney quali Topolino e Paperino e i fumetti Bonelli (che fra gli italiani
#sono probabilmente i più gettonati)

DFfilteredmagazineitems = DFfilteredbindingitems.filter(~(DFfilteredbindingitems.title.contains("Topolino")) | (DFfilteredbindingitems.title.contains("Paperino"))|(DFfilteredbindingitems.title.contains("Tex"))|(DFfilteredbindingitems.title.contains("Dylan Dog"))|(DFfilteredbindingitems.title.contains("Nathan Never"))|(DFfilteredbindingitems.title.contains("Zagor")))
#DFfilteredmagazineitems.show(20, False)

#FILTRAGGIO 4: libri con più di X votes. Per vedere i voti di ogni libro dovrei usare il dataframe link_person_item.csv.
#Come possiamo notare, alcune recensione hanno valore 0: questo non vuol dire che il libro è stato valutato
#atrocemente ma che l'utente ha letto il libro senza inserire voti.

#Dobbiamo dunque scremare tali voti da questo dataframe.

# file = "/data/SMARTDATA/books/anobii/link_person_item.csv" #MOSTRA IL VOTO DI UN UTENTE A UN LIBRO
# DFstars = spark.read.csv(file,header=True)
#DFstars.show(50, False)

DFstarsfilteredno0 = DFstars.filter(DFstars.item_review > 0)
#DFstarsfilteredno0.show(20,False)

#Contiamo i libri con più voti e visualizziamoli in ordine discendente

DFgrouped = DFstarsfilteredno0.groupby("item_id").count().withColumnRenamed("count", "total_votes").sort("total_votes", ascending=False)
#I voti dei libri più votati si aggirano intorno ai 10^4 come ordine di grandezza per cui per ora scelgo 300 come
#soglia di voti necessaria a rimanere nel dataframe (soggetto a cambiamenti)
DFgroupedfiltered = DFgrouped.filter(DFgrouped['total_votes'] > 300)
DFjoinbookstars = DFfilteredmagazineitems.join(DFgroupedfiltered, DFfilteredmagazineitems.item_id == DFgroupedfiltered.item_id).drop(DFgroupedfiltered.item_id)

#FILTRAGGIO 5: Pulizie varie ed eventuali
#DFfilteredlessthan10items.filter(DFfilteredlessthan10items.total_review > 100).show()

#NOTA: prendo i titoli con le minuscole per facilità di merging con le biblioteche
DFmanifestations_definitive_anobii = DFjoinbookstars.select("item_id", lower(DFjoinbookstars.title), lower(DFjoinbookstars.sub_title), \
                                                            "isbn", "average_rating", "total_review", "total_wishlist", \
                                                            DFjoinbookstars['total_votes']).withColumnRenamed("lower(title)", "title").withColumnRenamed("lower(sub_title)", "sub_title")
#remove rows with null in ibsn
# DFmanifestations_definitive_anobii.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["isbn"]]).show() #--> sono 47: togliamoli !!!
DFmanifestations_definitive_anobii = DFmanifestations_definitive_anobii.na.drop(subset=["isbn"])
# DFmanifestations_definitive_anobii.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["isbn"]]).show() #--> sono zero!
# DFitems_anobii.select([count(when(isnan(c) | isnull(c), c)).alias(c) for c in ["isbn"]]).show()

In [3]:
# DFmanifestations_definitive_anobii.show(1, False)
DFfiltered = DFjoinbookstars.select("item_id", lower(DFjoinbookstars.title), \
                                    lower(DFjoinbookstars.sub_title), "isbn", "average_rating", "total_review", "total_wishlist", \
                                    DFjoinbookstars['total_votes']).withColumnRenamed("lower(title)", "title").withColumnRenamed("lower(sub_title)", "sub_title")
DFauthorsandbooks = DFdisplay.join(DFauthorbooks, DFdisplay.author_id == DFauthorbooks.author_id).drop(DFdisplay.author_id).filter(DFdisplay.language=="11")
DFvoteswithtitles = DFstarsfilteredno0.join(DFfiltered, DFfiltered.item_id == DFstarsfilteredno0.item_id).select(DFstarsfilteredno0.item_id, "person_id", "title", str("sub_title"), "item_review", "total_review", "average_rating", "total_votes", "isbn")
#DFvoteswithtitles.filter((DFvoteswithtitles.title == "il fuggiasco") & (DFvoteswithtitles.person_id == "262165")).show()
#DFvoteswithtitles.filter(DFvoteswithtitles.title == "la custode di libri").show()
#Per ora proverò a costruire una CSR (matrice sparsa compressa) dal dataset di anobii (senza join con le biblioteche,
#solo per testare). La CSR è costruita in modo tale da avere tre vettori numpy, uno per i dati e due per gli indici
#riga-colonna con cui poi ricostruire la matrice originale.

#Come posso passare dal dataset a questa matrice? La maniera più efficiente (ed è meglio adottarle dato le grandezze
#dei dataset) è quella di creare la matrice direttamente, senza prima crearne una vuota da riempire.

#Supposto che le righe di tale matrice siano gli utenti e le colonne siano i libri (e i dati siano per ora solo
#i voti, poi potremo aggiungere feature per i metadati) allora posso provare ad aggiungere al mio dataframe delle 
#colonne indicanti gli indici futuri di riga colonna, usando degli id incrementali.
DFvoteswithtitles = DFstarsfilteredno0.join(DFfiltered, DFfiltered.item_id == DFstarsfilteredno0.item_id).select(DFstarsfilteredno0.item_id, "person_id", "title", \
                                                                                                                 str("sub_title"), "item_review", "total_review", \
                                                                                                                 "average_rating", "total_votes", "isbn")
#Proviamo. (Forse con gli RDD è meglio)
RDDvoteswithtitles = DFvoteswithtitles.rdd.cache()
#RDDpair = RDDvoteswithtitles.map(lambda x: (x.person_id, list(x))).sortByKey()#.collect()
RDDmapped = RDDvoteswithtitles.map(addDataType1)
#user_dictionary = create_user_dictionary(RDDvoteswithtitles)
#book_dictionary = create_book_dictionary(RDDvoteswithtitles)
#print(RDDmapped.take(20))

DFwithindexes = RDDmapped.toDF(["item_id", "person_id", "title", "sub_title",  "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type"], sampleRatio = 0.9) 
#Per evitare il problema del sampling in sub_title
#DFvoteswithtitles.filter((DFvoteswithtitles.title == "il fuggiasco") & (DFvoteswithtitles.person_id == "262165")).show()
#DFwithindexes contiene i voti di ogni utente dato il libro, altre info utili e gli indici di riga e di colonna
#della eventuale compressed sparse matrix per una facile ed efficiente creazione (per i sistemi di raccomandazione)
#Fatto ciò, riprendiamo il database delle biblioteche filtrato (come si importa un altro file .ipynb?)

In [4]:
##INSERIAMO I NUOVI ATTRIBUTI (ENCRYPTED_ITEM_ID E GENRE DAI DATASET NUOVI)
DFwithindexes_encrypted  = DFwithindexes.join(DFnew_data, DFnew_data.item_id == DFwithindexes.item_id).drop(DFnew_data.item_id).drop(DFnew_data.sub_title).drop(DFnew_data.title).drop(DFnew_data.total_review).drop(DFnew_data.average_rating).drop(DFnew_data.isbn).select("item_id", "person_id", "title", "sub_title",  "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type", "encrypt_item_id")

DFwithindexes_genres = DFwithindexes_encrypted.join(DFgenres, DFgenres.itemid == DFwithindexes_encrypted.item_id).select("item_id", "person_id", "title", "sub_title",  "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type", "encrypt_item_id", "name")

#DFwithindexes_genres.show(20, False)

DFwithindexes_genres = DFwithindexes_genres.withColumnRenamed("name", "genre")

In [5]:
%run cleanDataBCTwithIBSN.ipynb #Così a quanto pare
DFmanifestations_definitive_bct = DFmanifestations_definitive
# DFmanifestations_definitive_bct.show(1, False)
# DFmanifestations_definitive_bct.count()

Togli oggetti inutili: Prima 290125  - Dopo 257054
Togli libri in lingua straniera: Prima 290125  - Dopo 228059
Togli Fumetti: Prima 290125  - Dopo 227749
prefiltrati: 218419 - filtrati: 17947
Manifestation prefiltrati: 218419 - filtrati: 18164
Item prefiltrati: 998403 - filtrati: 217405
Loan prefiltrati: 5484078 - filtrati: 2172971
i libri sono: 18164


In [5]:
DFwithindexes.show(1)

+-------+---------+-----------------+---------+-----------+------------+----------------+-----------+----------+---------+
|item_id|person_id|            title|sub_title|item_review|total_review|  average_rating|total_votes|      isbn|data_type|
+-------+---------+-----------------+---------+-----------+------------+----------------+-----------+----------+---------+
|1981748|    68170|giulietta squeenz|     None|          3|         128|3.20037105751391|        544|8845260372|   anobii|
+-------+---------+-----------------+---------+-----------+------------+----------------+-----------+----------+---------+
only showing top 1 row



In [6]:
#Per adesso, concentriamoci su DFloans_definitive (i prestiti scremati).

#DFauthorsandbooks.filter(DFauthorsandbooks.item_id == "1254828").show()
#DFauthorsandbooks.filter(DFauthorsandbooks.item_id=="651589").show(20, False)
DFwithindexes_and_authors = DFwithindexes_genres.join(DFauthorsandbooks, DFwithindexes.item_id == DFauthorsandbooks.item_id).\
                                          select(DFwithindexes.item_id, "person_id", "title", "sub_title", "author_name", \
                                                 "item_review", "total_review", "average_rating", "total_votes", "isbn", "data_type", "encrypt_item_id", "genre")
#Prendo i prestiti (che alla fine, è ciò che mi interessa) con i titoli che, per una formattazione più agevole, rendo minuscoli.
DFloans_with_titles = DFloans_definitive.join(DFmanifestations_definitive, DFloans_definitive.manifestation_id_new == DFmanifestations_definitive.manifestation_id_new)\
                                        .select(DFloans_definitive.manifestation_id_new, "patron_id_md5", "author", "ISBNISSN_new", \
                                                lower(DFmanifestations_definitive.title)).\
                                        withColumnRenamed("lower(title)", "title")
#DFwithindexes_and_authors.filter((DFwithindexes_and_authors.title == "il fuggiasco") & (DFwithindexes_and_authors.person_id == "262165")).show()
#Di cosa ho bisogno in loans_definitive per renderlo "simile" alle tuple di anobii? Certamente avrò bisogno del titolo, poi del manifestation_id (item_id), del cliente della biblioteca
#(orrispettivo di person_id), di assegnare una item_review provvisoria (abbiamo stabilito, almeno per ora, 4 NOTA: l'average rating lo setto a NULL nelle tuple o lo ricalcolo considerando le 
#biblioteche? Da vedere), di settare le total_review (per ora a NULL?), i total_votes, l'isbn e lo user e book index (a NULL?)

#Il problema di molti campi che lascerei a NULL è che in alcuni casi i titoli non sono esattamente gli stessi a causa della formattazione dei database delle biblioteche. Per stemperare il problema,
#proverò a scindere titolo e sottotitoli in due campi differenti (alla maniera di anobii) e a pulire eventuali simboli strani (ad esempio, i numeri all'inizio di alcuni titoli).

RDDloans_with_titles = DFloans_with_titles.rdd
    
RDDloans_for_merge = RDDloans_with_titles.map(title_and_subtitle)  

In [7]:
DFloans_with_titles.show(1, False)

+--------------------+--------------------------------+-------------+------------+---------+
|manifestation_id_new|patron_id_md5                   |author       |ISBNISSN_new|title    |
+--------------------+--------------------------------+-------------+------------+---------+
|107930              |5bd489802c61305cc115ad942fa60908|Carlo Collodi|8807820714  |pinocchio|
+--------------------+--------------------------------+-------------+------------+---------+
only showing top 1 row



In [7]:
DFloans_to_merge = RDDloans_for_merge.toDF(["item_id", "person_id", "title", "author_name", "sub_title", "item_review", \
                                            "total_review", "average_rating", "total_votes", "isbn", "data_type"])

from pyspark.sql.functions import lit
DFloans_to_merge = DFloans_to_merge.withColumn("encrypt_item_id", lit(None))
DFloans_to_merge = DFloans_to_merge.withColumn("genre", lit(None))

DFmerged = DFwithindexes_and_authors.union(DFloans_to_merge)
### MERGE AVVENUTO (sperando di averlo implementato bene...)

#Adesso vorrei cercare di sostituire i book_id delle biblioteche con quelli del corrispettivo titolo di anobii. Ciò aiuterà quando creerò la 
#colonna degli indici per la compressed sparse matrix.
#Inoltre, vorrei che la funzione aggiornasse le metriche delle tuple delle biblioteche, per ora cosparse di "None".

#Come discusso nel meeting, vorrei inoltre eliminare dal mio dataframe mergiato tutti quei titoli con una sola parola (che di solito si riferiscono a guide turistiche).

#Infine, vorrei inserire gli indici di colonna e di riga della compressed sparse matrix in modo da iniziare a preparare l'input per la libreria di raccomandazione 
#(ho già implementato le funzioni
#per farlo, a patto che le chiavi usate per il dizionario non siano erronee).

#FASE 1: i libri con lo stesso titolo dovrebbero avere lo stesso book_id (e le metriche dovrebbero essere aggiornate di conseguenza).
#Ciò è fattibile con la window di SQL e una partitionby. Controlliamo, con un sampling casuale, se ci sono due libri provenienti dai due diversi database (controllare data_type).

#DFmerged.filter(DFmerged.title == "la custode di libri").show() #TROVATO ERRORE DI RIPETIZIONE TUPLE A CAUSA DELLA MANCATA SCREMATURA DEL LINGUAGGIO DURANTE IL JOIN PER LE INFO SULL'AUTORE (SISTEMATO)

#ERRORE 2: alcuni libri di anobii sono ripetuti due volte con un autore diverso (ma perché?), da sistemare con un'aggregazione
#Prima, però. filtriamo via i titoli con una parola (dovrebbero essere pochi rispetto alla cardinalità totale e ci semplifica la vita con i libri omonimi)
#print(DFmerged.count()) #CARDINALITà DATABASE MERGIATO COUNT = 10331428
DFmerged_no_onewordtitles = DFmerged.filter(DFmerged.title.contains(" ")) #Se contiene uno spazio allora non è una sola parola (sperando di aver formattato bene)
#DFmerged_no_onewordtitles.filter(DFmerged_no_onewordtitles.title == "barcellona").show()
#DFmerged.filter(DFmerged.title == "barcellona").show()

#Procediamo adesso con la sostituzione dei book_id delle biblioteche con quelli di anobii
DFmerged_aggregated = DFmerged_no_onewordtitles.select("*", F.min(DFmerged_no_onewordtitles.item_id).over(Window.partitionBy(DFmerged_no_onewordtitles.title)).alias("book_id"))
### MERGE AVVENUTO (sperando di averlo implementato bene...)

#Adesso vorrei cercare di sostituire i book_id delle biblioteche con quelli del corrispettivo titolo di anobii. Ciò aiuterà quando creerò la colonna degli indici per la compressed sparse matrix.
#Inoltre, vorrei che la funzione aggiornasse le metriche delle tuple delle biblioteche, per ora cosparse di "None".

#Come discusso nel meeting, vorrei inoltre eliminare dal mio dataframe mergiato tutti quei titoli con una sola parola (che di solito si riferiscono a guide turistiche).

#Infine, vorrei inserire gli indici di colonna e di riga della compressed sparse matrix in modo da iniziare a preparare l'input per la libreria di raccomandazione (ho già implementato le funzioni
#per farlo, a patto che le chiavi usate per il dizionario non siano erronee).

#FASE 1: i libri con lo stesso titolo dovrebbero avere lo stesso book_id (e le metriche dovrebbero essere aggiornate di conseguenza).
#Ciò è fattibile con la window di SQL e una partitionby. Controlliamo, con un sampling casuale, se ci sono due libri provenienti dai due diversi database (controllare data_type).

#DFmerged.filter(DFmerged.title == "la custode di libri").show() #TROVATO ERRORE DI RIPETIZIONE TUPLE A CAUSA DELLA MANCATA SCREMATURA DEL LINGUAGGIO DURANTE IL JOIN PER LE INFO SULL'AUTORE (SISTEMATO)

#ERRORE 2: alcuni libri di anobii sono ripetuti due volte con un autore diverso (ma perché?), da sistemare con un'aggregazione
#Prima, però. filtriamo via i titoli con una parola (dovrebbero essere pochi rispetto alla cardinalità totale e ci semplifica la vita con i libri omonimi)
#print(DFmerged.count()) #CARDINALITà DATABASE MERGIATO COUNT = 10331428
DFmerged_no_onewordtitles = DFmerged.filter(DFmerged.title.contains(" ")) #Se contiene uno spazio allora non è una sola parola (sperando di aver formattato bene)
#DFmerged_no_onewordtitles.filter(DFmerged_no_onewordtitles.title == "barcellona").show()
#DFmerged.filter(DFmerged.title == "barcellona").show()

#Procediamo adesso con la sostituzione dei book_id delle biblioteche con quelli di anobii
DFmerged_aggregated = DFmerged_no_onewordtitles.select("*", F.min(DFmerged_no_onewordtitles.item_id).over(Window.partitionBy(DFmerged_no_onewordtitles.title)).alias("book_id"))
#print(DFmerged_no_onewordtitles.count()) #DOPO L'ELIMINAZIONE DEI LIBRI CON UNA SOLA PAROLA COUNT = 8895429

#Aggreghiamo adesso ripetizioni di uno stessa coppia (item_id, user_id) (PER ELIMINARE LE TUPLE RIPETUTE, VEDESI ERRORE 2 DOVUTO ALLA FORMATTAZIONE DI ANOBII)
#DFmerged_aggregated.filter(DFmerged_aggregated.title == "la custode di libri").show()

#Troviamo "new_author" formattato in maniera coerente fra i database mergiati
DFmerged_aggregated_norepetition = DFmerged_aggregated.select("*", F.min(DFmerged_aggregated.author_name).over(Window.partitionBy(DFmerged_aggregated.book_id)).alias("new_author"))
#DFmerged_aggregated_norepetition.filter(DFmerged_aggregated_norepetition.title == "la custode di libri").show()

#Aggreghiamo anche i generi e l'encrypt_item_id


#Infine, eliminiamo le ridondanze con dropDuplicates.
DFmerged_aggregated_genres = DFmerged_aggregated_norepetition.select("*", F.min(DFmerged_aggregated.genre).over(Window.partitionBy(DFmerged_aggregated.book_id)).alias("new_genre"))
DFmerged_aggregated_encrypt = DFmerged_aggregated_genres.select("*", F.min(DFmerged_aggregated.encrypt_item_id).over(Window.partitionBy(DFmerged_aggregated.book_id)).alias("new_encrypt"))            
                                                        
                                                              
DFmerged_aggregated_norepetition_unique = DFmerged_aggregated_encrypt.dropDuplicates(['book_id', 'person_id'])
#DFmerged_aggregated_norepetition_unique.filter(DFmerged_aggregated_norepetition_unique.title == "la custode di libri").show()

#Infine, droppiamo gli attributi che non servono più e rinominiamo quelli nuovi e più utili
DFmerged_final = DFmerged_aggregated_norepetition_unique.drop("item_id").drop("author_name").drop("encrypt_item_id").drop("genre")
#ERRORE 2 SISTEMATO
#DFmerged_final.filter(DFmerged_final.book_id == "1254828").show()
#DFmerged_final.orderBy(rand()).show(50, False)
#Adesso che il merge e gli attributi sono un po' più solidi possiamo provare a calcolare qualche metrica anche per le tuple della biblioteca
#Prima però vorrei controllare una cosa

#DFmerged_final.filter(DFmerged_final.title == "1 ora").show()
#DFmerged_final.filter(DFmerged_final.title == "un'ora").show()

#Ho provato superficialmente a considerare libri con un numero nel titolo per controllare se fossero formattati diversamente, continuerò il controllo più tardi
#Una metrica da considerare per le tuple "biblioteche" potrebbe essere il numero di prestiti per un certo libro (sapere che un prestito è avvenuto 20 o 2000 volte è certamente un'info notevole)

#Nota: sarebbe consono anche aggiornare le metriche di anobii post raggruppamento
DFmerged_count = DFmerged_final.select("*", F.count("*").over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("total_count"))
#DFmerged_count.orderBy(rand()).show(50, False)
DFmerged_average = DFmerged_count.select("*", F.avg(DFmerged_final.item_review).over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("new_average_rating"))
DFmerged_maxreviews = DFmerged_average.select("*", F.max(DFmerged_final.total_review).over(Window.partitionBy(DFmerged_final.book_id, DFmerged_final.data_type)).alias("new_total_review"))

#NOTA: l'average rating per le tuple delle biblioteche sarà, almeno per ora, ovviamente 4.0 per via dei voti fissi. new_number_reviews rappresenta il numero massimo di recensioni in caso di incongruenze.
#DFmerged_sumreviews.orderBy(rand()).show(50, False)

#Infine, inseriamo l'isbn anche per le tuple delle biblioteche  che abbiano una corrispondenza in anobii(potrebbe servire ed è anche utile per ricercare eventuali libri dalla formattazione errata)
DFmerged_isbn = DFmerged_maxreviews.select("*", F.min(DFmerged_maxreviews.isbn).over(Window.partitionBy(DFmerged_final.book_id)).alias("new_isbn"))
#DFmerged_isbn.orderBy(rand()).show(20, False)  

#DFmerged_isbn.orderBy(rand()).filter(DFmerged_isbn.title == "i giorni dell'arcobaleno").show(50) #LIBRO PRESENTE SOLO NELLE BIBLIOTECHE TROVATO CON ISBN == None

#Adesso droppiamo gli attributi inutili e rinominiamo quelli utili

DFmerged_almost_definitive = DFmerged_isbn.drop("total_review").drop("total_votes").drop("average_rating").drop("isbn")\
                                          .withColumnRenamed("total_count", "total_votes_or_loans")\
                                          .withColumnRenamed("new_average_rating", "average_rating")\
                                          .withColumnRenamed("new_isbn", "isbn")\
                                          .withColumnRenamed("new_total_review", "total_review")\
                                          .withColumnRenamed("new_author", "author")\
                                          .withColumnRenamed("new_genre", "genre")\
                                          .withColumnRenamed("new_encrypt", "encrypt_item_id")
                                            

#DFmerged_almost_definitive.show(1, False)
#Adesso è il momento di attaccare gli indici di riga e colonna per la compressed sparse matrix tramite le funzioni che ho creato qualche cella fa.

#DFmerged_almost_definitive.filter(typeof(DFmerged_almost_definitive.person_id) != str).show()
RDDmerged_almost_definitive = DFmerged_almost_definitive.rdd
#from itertools import islice

#def take(n, iterable):
#    "Return first n items of the iterable as a list"
#    return list(islice(iterable, n))
book_dictionary = create_book_dictionary(RDDmerged_almost_definitive)
user_dictionary = create_user_dictionary(RDDmerged_almost_definitive)
#n_items = take(20, user_dictionary.items())
RDDdefinitive = RDDmerged_almost_definitive.map(addRowColumnId)
#print(n_items)
#print(RDDdefinitive.take(1))
DFdefinitive = RDDdefinitive.toDF(["person_id", "title", "sub_title", "item_review", "data_type", "book_id", "author", "genre", "encrypt_item_id", "total_votes_or_loans",\
                                   "average_rating", "total_review", "isbn", "user_index", "book_index"], sampleRatio=0.9)
#DFdefinitive.show(1, False) ###COLLABORATIVE FILTERING###
#Merge per raccomandazione content based
print(DFdefinitive.count()) #COUNT 5266964
print(DFdefinitive.filter(DFdefinitive.genre!="null").count()) #COUNT 
#Ho bisogno di un dataframe che contenga tutti i libri (dei dataset anobii e biblioteche) presenti post-filtraggio.
#Per prima cosa, carichiamo DFmanifestations_definitive da cleanedData. (per eseguire cleanedData, vedere qualche cella sopra).

#Come è fatto DFmanifestations_definitive?

#DFmanifestations_definitive.show(10, False)
# DFmanifestations_definitive.filter(DFmanifestations_definitive.manifestation_id_new == "107930").show()

#Come possiamo vedere, il dataframe contiene i libri, presi una singola volta, secondo lo schema:
#edition_language, edition_date, title, author, publisher, manifestation_id_new (che è il book_id).

#E invece anobii? Per vedere i libri di anobii dobbiamo controllare DFitems_anobii.

#DFitems_anobii.show(1, False)

#NOTA: edition_date e edition_language di DFmanifestations_definitive sono eliminabili poiché non utili ai fini della raccomandazione
#NOTA2: devo dividere titolo e sottotitolo come ho fatto per l'altro merge, oltre a rendere minuscoli tutti i caratteri
#NOTA 3: il publisher deve rimanere per entrambi i dataset (e magari renderli coerenti)

#Usiamo il dataset filtrato di DFitems_anobii: DFjoinbookstars (ha senso mantenere il filtraggio fatto rispetto ai numeri di voti in un content based? Da vedere)

#DFjoinbookstars.show(1, False)

5266964
0


In [8]:
#DFdefinitive.filter((DFdefinitive.title.contains("harry potter")) & (DFdefinitive.data_type =="biblioteche")).show(10, False)

+--------------------------------+------------------------------------+---------+-----------+-----------+-------+-------------+--------+------------------+--------------------+--------------+------------+----------+----------+----------+
|person_id                       |title                               |sub_title|item_review|data_type  |book_id|author       |genre   |encrypt_item_id   |total_votes_or_loans|average_rating|total_review|isbn      |user_index|book_index|
+--------------------------------+------------------------------------+---------+-----------+-----------+-------+-------------+--------+------------------+--------------------+--------------+------------+----------+----------+----------+
|0057b060eea06ee994b98c422b97f473|harry potter e la camera dei segreti|None     |4          |biblioteche|105503 |J. K. Rowling|Children|010990eaa2a97d8b83|793                 |4.0           |None        |0828877157|159       |176       |
|0059a7e6d21805f9ebc6d58b5aa6042f|harry potter e

In [9]:
RDDmanifestations_definitive = DFmanifestations_definitive.rdd

RDDmanifestations_trimmed = RDDmanifestations_definitive.map(title_and_subtitle_books)

DFmanifestations_trimmed = RDDmanifestations_trimmed.toDF(["title", "sub_title", "author", "publisher", "book_id", "isbn"])
DFmanifestations_lowercase = DFmanifestations_trimmed.select(lower(DFmanifestations_trimmed.title), lower(DFmanifestations_trimmed.sub_title), "author", "publisher", "book_id", "isbn")\
                                                     .withColumnRenamed("lower(title)", "title")\
                                                     .withColumnRenamed("lower(sub_title)", "sub_title")

#DFmanifestations_lowercase.show(1, False)

#NOTA: eliminiamo i libri con una sola parola nel titolo per ragioni analoghe a quelle del merge collaborativo
DFmanifestations_nooneword = DFmanifestations_lowercase.filter(DFmanifestations_lowercase.title.contains(" "))
# DFmanifestations_nooneword.show(1, False)

#PER I NUOVI ATTRIBUTI
DFmanifestations_nooneword = DFmanifestations_nooneword.withColumn("encrypt_item_id", lit(None))
DFmanifestations_nooneword = DFmanifestations_nooneword.withColumn("genre", lit(None))
DFmanifestations_nooneword = DFmanifestations_nooneword.select("title", "sub_title", "author", "publisher", "encrypt_item_id", "book_id", "isbn", "genre")
#print(DFmanifestations_lowercase.count()) #COUNT 18164
#print(DFmanifestations_nooneword.count()) #COUNT 16143

#Adesso, trimmiamo DFjoinbookstars.
#NOTA: abbiamo bisogno dell'autore quindi dobbiamo fare un join con DFauthorsandbooks (vedere celle sopra)

DFjoinbookstars_and_authors = DFjoinbookstars.join(DFauthorsandbooks, DFjoinbookstars.item_id == DFauthorsandbooks.item_id)\
                                             .select(lower(DFjoinbookstars.title), lower(DFjoinbookstars.sub_title), "author_name", "publisher", "isbn", DFjoinbookstars.item_id, )\
                                             .withColumnRenamed("author_name", "author")\
                                             .withColumnRenamed("item_id", "book_id")\
                                             .withColumnRenamed("lower(title)", "title")\
                                             .withColumnRenamed("lower(sub_title)", "sub_title")
DFjoinbookstars_and_authors_nooneword = DFjoinbookstars_and_authors.filter(DFjoinbookstars_and_authors.title.contains(" "))
# DFjoinbookstars_and_authors_nooneword.show(1, False)

#print(DFjoinbookstars_and_authors.count()) #COUNT 8524
#print(DFjoinbookstars_and_authors_nooneword.count()) #COUNT 7273

In [10]:
#Fatto ciò, joiniamo questo dataset con quello dei libri finali (non DFbooks_final ma DFjoinbookstars_and_authors_nooneword
#che è il dataset prima del merge). Dobbiamo quindi posticipare il merge con le biblioteche.

DFmanifestations_with_encrypt = DFjoinbookstars_and_authors_nooneword.join(DFnew_data, DFnew_data.item_id == DFjoinbookstars_and_authors_nooneword.book_id).drop(DFnew_data.title).drop(DFnew_data.sub_title).drop(DFnew_data.publisher).drop(DFnew_data.isbn).select("title", "sub_title", "author", "publisher", "encrypt_item_id", "book_id", "isbn")
#DFmanifestations_with_encrypt.filter(DFmanifestations_with_encrypt.book_id == "1981748").show(10, False)

#Fatto ciò, inseriamo anche il genere usando il file csv da me creato dal file item_category.sql 
#NOTA: avendolo dovuto rifare da zero per via di corruzioni del file originale è possibile ci siano delle righe
#di rumore, eliminarle.
#NOTA2: aggiungere new_genres.csv ad Hadoop. (FATTO)

#Adesso, joiniamo usando item_id per ottenere il genere nelle tuple dei libri (più informazioni per il sistema di
#raccomandazione).

#NOTA:l'item_id nel DFgenres compare più volte. Questo immagina sia dovuto alla presenza di più generi su un solo libro.
#Cercherò adesso di combinarli in un genere solo tramite una funzione RDD.

RDDgenres = DFgenres.rdd
RDDgenres.sortBy(lambda x: x.name) #Per avere i generi ordinati alfabeticamente
RDDgenres_keyvalue = RDDgenres.map(lambda x: (x.itemid, list(x)))
#print(RDDgenres_keyvalue.take(1))

def reduceGenres(x, y): #Ritorna il libro con i generi accorpati
    return [x[0], x[1], x[2], x[3], x[4] + " / " + y[4], x[5] + " / " + y[5], x[6], x[7]]

RDDreduced = RDDgenres_keyvalue.reduceByKey(reduceGenres)
#print(RDDreduced.take(1))

#La funzione di reduce sembra funzionare. Riportiamolo in forma di RDD singolo e poi di dataframe.
def toSingle(line):
    return Row(str(line[1][0]), str(line[1][1]), str(line[1][2]), str(line[1][3]), str(line[1][4]), str(line[1][5]), str(line[1][6]), str(line[1][7]))
RDDsingle = RDDreduced.map(toSingle)
#print(RDDsingle.take(20))
DFgenres_agg = RDDsingle.toDF(['id', 'familyid', 'book_id', 'categoryid', 'slug', 'genre', 'language', 'votes'])
#DFgenres_agg.show(1, False)
#NOTA2: bisogna anche raggruppare le ripetizioni dei libri dovute ai diversi autori
#Questo tipo di aggregazione si può fare (per ora) con una aggregazione partitionBy
#NOTA PER IL FUTURO: implementare funzione per aggregare gli autori in maniera simile alla funzione
#reduceGenres per evitare l'aggregazione per ordine alfabetico (ciò vale anche per l'altra aggregazione per autore)
#DFmanifestations_with_encrypt.filter(DFmanifestations_with_encrypt.book_id=="2421303").show()
DFmanifestations_with_encrypt_agg_author = DFmanifestations_with_encrypt.select("*", F.min(DFmanifestations_with_encrypt.author).over(Window.partitionBy(DFmanifestations_with_encrypt.book_id)).alias("new_author")).drop("author").withColumnRenamed("new_author", "author").dropDuplicates(['book_id', 'author'])
DFbooks_with_genres = DFmanifestations_with_encrypt_agg_author.join(DFgenres_agg, DFgenres_agg.book_id == DFmanifestations_with_encrypt_agg_author.book_id).drop(DFmanifestations_with_encrypt_agg_author.book_id).select("title", "sub_title", "author", "publisher", "encrypt_item_id", "book_id", "isbn", "genre")
#DFbooks_with_genres.show(20, False)
#DFbooks_with_genres.filter(DFbooks_with_genres.title=="il mercante di quadri scomparsi").show()
#Possiamo adesso procedere con il merge

In [11]:
#Gli attributi sono gli stessi: procediamo al merge

DFcontentbased_merge = DFmanifestations_nooneword.union(DFbooks_with_genres)

#Adesso dobbiamo raggruppare sotto lo stesso id i libri uguali ma provenienti da due dataset diversi

DFbooks_sameisbn = DFcontentbased_merge.select("*", F.min(DFcontentbased_merge.isbn).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_isbn"))
DFbooks_sameid = DFbooks_sameisbn.select("*", F.min(DFcontentbased_merge.book_id).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_book_id"))
#Approfittiamone per raggruppare anche autori e publisher (per rendere le tuple più coerenti) (serve?)
DFbooks_samepublisher = DFbooks_sameid.select("*", F.min(DFcontentbased_merge.publisher).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_publisher"))
DFbooks_sameauthor = DFbooks_samepublisher.select("*", F.min(DFcontentbased_merge.author).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_author"))
#E raggruppiamo anche il genere e l'encrypted_item_id
DFbooks_samegenre = DFbooks_sameauthor.select("*", F.min(DFcontentbased_merge.genre).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_genre"))
DFbooks_sameencrypted = DFbooks_samegenre.select("*", F.min(DFcontentbased_merge.encrypt_item_id).over(Window.partitionBy(DFcontentbased_merge.title)).alias("new_encrypted"))
#Droppiamo i vecchi attributi e lasciamo i nuovi

DFbooks_dropid = DFbooks_sameencrypted.drop("book_id").drop("publisher").drop("author").drop("encrypt_item_id").drop("genre").drop("isbn")\
                                   .withColumnRenamed("new_book_id", "book_id")\
                                   .withColumnRenamed("new_author", "author")\
                                   .withColumnRenamed("new_publisher", "publisher")\
                                   .withColumnRenamed("new_isbn", "isbn")\
                                   .withColumnRenamed("new_genre", "genre")\
                                   .withColumnRenamed("new_encrypted", "encrypt_item_id")
#Adesso droppiamo i duplicati

DFbooks_final = DFbooks_dropid.dropDuplicates(["book_id", "author", "publisher"])

#DFbooks_final.orderBy(rand()).show(100, False)
#DFbooks_final.filter(DFbooks_final.title=="l'eleganza del riccio").show()
#print(DFbooks_final.count()) #COUNT 16329
#print(DFbooks_final.filter(DFbooks_final.genre!=None).count()) #COUNT 2782

In [95]:
#Salviamo DFbooks_final con i nuovi attributi in un file csv
#DFbooks_final.toPandas().to_csv("books_with_new_attributes.csv")

In [19]:
books = DFbooks_final
# books.filter(books.title == "il fiore di vincenzo").show(1, False)
ratings = DFdefinitive
# ratings.filter(ratings.title == "il fiore di vincenzo").show(1, False)

#take data in ratings that must be placed in books table
isnbnForbook_id = ratings.dropDuplicates(["book_id"])
# isnbnForbook_id.show(1, False)
books_tocsv = books.join(isnbnForbook_id, books.book_id == isnbnForbook_id.book_id).select(books["*"],\
                                                                             isnbnForbook_id["data_type"], 
                                                                             isnbnForbook_id["total_votes_or_loans"],
                                                                             isnbnForbook_id["average_rating"],
                                                                             isnbnForbook_id["total_review"],
                                                                             isnbnForbook_id["book_index"])
#drop books which has null isbn
books_tocsv = books_tocsv.filter(books_tocsv.isbn.isNotNull()).na.drop(subset=["isbn"])
books_tocsv.show(1, False)
books_tocsv.toPandas().to_csv("books.csv")
print("there are "+str(books_tocsv.count())+" books")

toDrop = [#"person_id",
          "title",
          "sub_title",
          "data_type",
          "author",
          "total_votes_or_loans",
          "average_rating",
          "total_review",
          "isbn",
          "genre", 
          "encrypt_item_id",
          "user_index",
          "book_index",
          ]
ratings_tocsv = ratings.drop(*toDrop)
# ratings_tocsv.show(1, False)
ratings_tocsv = ratings_tocsv.join(books_tocsv, books_tocsv.book_id == ratings.book_id, "leftsemi")
ratings_tocsv.show(1, False)
ratings_tocsv.toPandas().to_csv("ratings.csv")
print("there are "+str(ratings_tocsv.count())+" rates")

users_tocsv = ratings.select("person_id", "user_index").dropDuplicates()
users_tocsv.show(1, False)
users_tocsv.toPandas().to_csv("users.csv")
print("there are "+str(users_tocsv.count())+" users")

+--------------------+----------------------------+----------+-------+---------+-------------------+-----+---------------+-----------+--------------------+--------------+------------+----------+
|title               |sub_title                   |isbn      |book_id|publisher|author             |genre|encrypt_item_id|data_type  |total_votes_or_loans|average_rating|total_review|book_index|
+--------------------+----------------------------+----------+-------+---------+-------------------+-----+---------------+-----------+--------------------+--------------+------------+----------+
|il fiore di vincenzo|storia di un pittore in erba|8882032450|125592 |Nord-Sud |Valerij H. Horbačov|null |null           |biblioteche|68                  |4.0           |None        |803       |
+--------------------+----------------------------+----------+-------+---------+-------------------+-----+---------------+-----------+--------------------+--------------+------------+----------+
only showing top 1 row

t

In [20]:
##############################
#  write file for doing API  #
#google book requests (local)#
##############################
id2isbn = books_tocsv.select(['book_id', 'isbn']).rdd.map(lambda row : (row[0],row[1])).collectAsMap()
with open('id2isbn.csv', 'w') as f:
    for key in id2isbn.keys():
        f.write("%s, %s\n" % (key, id2isbn[key]))

In [21]:
print(books.count()) #16329
print(books.filter(books.genre!= "null").count()) #2782


16329
2782


In [25]:
#books_tocsv.filter((books_tocsv.genre!="null") & (books_tocsv.data_type == "biblioteche")).show(10, False)

#NOTA: forse è meglio droppare i duplicati che contengono "anobii" e non quelli delle biblioteche, per avere
#un'idea di quanti libri delle biblioteche abbiano il genere

ratings.filter((ratings.genre!="null") & (ratings.data_type == "biblioteche")).show(10, False)

#Infatti in ratings questo tipo di caso è presente

+--------------------------------+-----------------------+---------+-----------+-----------+-------+---------------+-----+------------------+--------------------+--------------+------------+----------+----------+----------+
|person_id                       |title                  |sub_title|item_review|data_type  |book_id|author         |genre|encrypt_item_id   |total_votes_or_loans|average_rating|total_review|isbn      |user_index|book_index|
+--------------------------------+-----------------------+---------+-----------+-----------+-------+---------------+-----+------------------+--------------------+--------------+------------+----------+----------+----------+
|0035874fe0c316bce3db526c79840c0b|la variante di lüneburg|None     |4          |biblioteche|18726  |Paolo Maurensig|Crime|016eecb4d5426b7dcb|137                 |4.0           |None        |8845909840|89        |2705      |
|069c8bd97ef5e01185405e2a1740a300|la variante di lüneburg|None     |4          |biblioteche|18726  |Paol